## Data Pre Processing

In [17]:
# Import libraries
import pandas as pd

# Load CSV files
demographic_data = pd.read_csv('D:/OSNA/Project/NJ7_demogrphics.csv')
voter_data = pd.read_csv('D:/OSNA/Project/elections-2024-11-01T12_27_30.csv')
polling_data = pd.read_csv('D:/OSNA/Project/nj_polling_data.csv')

# Show the first few rows to verify data
print("Demographic Data:")
print(demographic_data.head())

print("\nVoter Data:")
print(voter_data.head())

print("\nPolling Data:")
print(polling_data.head())


Demographic Data:
    Topic      Subject             Title District 07 Estimate District 07 MOE
0  People  Sex and Age  Total population             7,78,678     (+/-11,305)
1  People  Sex and Age              Male             3,87,973      (+/-7,636)
2  People  Sex and Age            Female             3,90,705      (+/-6,631)
3  People  Sex and Age     Under 5 years               35,883      (+/-2,688)
4  People  Sex and Age      5 to 9 years               43,111      (+/-3,323)

Voter Data:
  candidate_id  candidate_election_year                candidate_name  \
0    P00000679                     2024           CARROLL, JERRY LEON   
1    P00002592                     2024             BENTLEY, THOMAS A   
2    P00003244                     2024  WINTERBOTTOM, THOMAS FRANCIS   
3    P00003814                     2024      MALONE, YEHANNA JOAN M M   
4    P00004861                     2024          BICKELMEYER, MICHAEL   

         party_full incumbent_challenge_full office  total_rec

In [23]:
import geopandas as gpd

# Read the shapefile
past_election_data = gpd.read_file("C:/Users/finof/Downloads/nj_vest_20_vtd_estimates/nj_vest_20_vtd_estimates.shp")

# Show the first few rows to verify data
print("\nPast Election Data (Shapefile):")
print(past_election_data.head())



Past Election Data (Shapefile):
  CONGR_DIST LEGIS_DIST COCOM_DIST       ELECD_KEY    COUNTY  \
0         02         02         05  001-0117-01-00  Atlantic   
1         02         02         05  001-0105-00-02  Atlantic   
2         02         02         05  001-0105-00-01  Atlantic   
3         02         02         05  001-0105-00-03  Atlantic   
4         02         02         02  001-0116-00-03  Atlantic   

               MUN_NAME MCODE_SVRS MCODE_1040 WARD_CODE ELECD_CODE  \
0      Mullica Township       0117       0117        01         00   
1  Buena Vista Township       0105       0105        00         02   
2  Buena Vista Township       0105       0105        00         01   
3  Buena Vista Township       0105       0105        00         03   
4          Margate City       0116       0116        00         03   

   G20PREDBID  G20PRERTRU  G20PRELJOR  G20PREGHAW  G20PRECBLA  G20PRESLAR  \
0         490         416           6           1           1           1   
1      

In [28]:
import pandas as pd

# Load the demographic data
demographic_data = pd.read_csv('D:/OSNA/Project/NJ7_demogrphics.csv')

# Fill or drop NaN values in 'District 07 Estimate' column (we'll drop NaN values here)
demographic_data = demographic_data.dropna(subset=['District 07 Estimate'])

# Remove commas in 'District 07 Estimate'
demographic_data['District 07 Estimate'] = demographic_data['District 07 Estimate'].replace({',': ''}, regex=True)

# Now, we will convert to numeric, coercing errors into NaN
demographic_data['District 07 Estimate'] = pd.to_numeric(demographic_data['District 07 Estimate'], errors='coerce')

# Drop rows where 'District 07 Estimate' is now NaN
demographic_data = demographic_data.dropna(subset=['District 07 Estimate'])

# Print cleaned data
print(demographic_data[['Topic', 'District 07 Estimate']].head())


    Topic  District 07 Estimate
0  People              778678.0
1  People              387973.0
2  People              390705.0
3  People               35883.0
4  People               43111.0


In [25]:
import geopandas as gpd
import pandas as pd

# Load the shapefile data (election precincts data)
past_election_data = gpd.read_file('C:/Users/finof/Downloads/nj_vest_20_vtd_estimates/nj_vest_20_vtd_estimates.shp')

# Show basic info about the shapefile (including the geometry)
print(past_election_data.info())

# Clean up the relevant columns for merging with other datasets
past_election_data_cleaned = past_election_data[['CONGR_DIST', 'COUNTY', 'MUN_NAME', 'ELECD_KEY', 'G20PREDBID', 'G20PRERTRU', 'geometry']]

# Let's also clean up the `COUNTY` and `MUN_NAME` columns (make them consistent for merging)
past_election_data_cleaned['COUNTY'] = past_election_data_cleaned['COUNTY'].str.strip()
past_election_data_cleaned['MUN_NAME'] = past_election_data_cleaned['MUN_NAME'].str.strip()

# Merge with demographic data based on `COUNTY` (or you can also merge based on `MUN_NAME` if needed)
# Assuming demographic_data has a COUNTY column, we will merge it based on that.
merged_data = pd.merge(past_election_data_cleaned, demographic_data, left_on='COUNTY', right_on='Topic', how='left')

# Check merged data
print(merged_data.head())

# Now let's merge the voter data, focusing on the `candidate_name` and `party_full` columns
# Assuming 'candidate_name' in voter_data corresponds with candidate columns in election data
merged_data = pd.merge(merged_data, voter_data_cleaned[['candidate_name', 'party_full', 'total_receipts']], on='candidate_name', how='left')

# Show the merged data
print(merged_data.head())

# Visualize data with geopandas (optional: if needed for your analysis)
merged_data.plot(column='G20PREDBID', legend=True)  # Example: Visualizing Biden's votes from 2020


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6349 entries, 0 to 6348
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   CONGR_DIST  6349 non-null   object  
 1   LEGIS_DIST  6349 non-null   object  
 2   COCOM_DIST  6349 non-null   object  
 3   ELECD_KEY   6349 non-null   object  
 4   COUNTY      6349 non-null   object  
 5   MUN_NAME    6349 non-null   object  
 6   MCODE_SVRS  6349 non-null   object  
 7   MCODE_1040  6349 non-null   object  
 8   WARD_CODE   6349 non-null   object  
 9   ELECD_CODE  6349 non-null   object  
 10  G20PREDBID  6349 non-null   int64   
 11  G20PRERTRU  6349 non-null   int64   
 12  G20PRELJOR  6349 non-null   int64   
 13  G20PREGHAW  6349 non-null   int64   
 14  G20PRECBLA  6349 non-null   int64   
 15  G20PRESLAR  6349 non-null   int64   
 16  G20PREOHAM  6349 non-null   int64   
 17  G20PREOFUE  6349 non-null   int64   
 18  G20USSDBOO  6349 non-null   int64   
 19

C:\Users\finof\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


NameError: name 'voter_data_cleaned' is not defined